1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score, accuracy_score

from sklearn.preprocessing import MinMaxScaler

In [58]:
df = pd.read_csv('train_case2.csv', sep=';')

In [59]:
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


Видно, что нужно поработать с возрастом потому что значения сильно отличаются от остальных. Нужно подключить скалер.

In [60]:
target = df.cardio
df = df.drop(['cardio'], 1)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 6.4 MB


Пропусков нету, все признаки непрерывные. Это хорошо. Не надо работать с данными.

In [62]:
scaler = MinMaxScaler()
cols = df.columns
df = pd.DataFrame(
    scaler.fit_transform(df.to_numpy()),
    columns=cols
                 )
df

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,0.00000,0.588076,1.0,0.579487,0.273684,0.016079,0.013550,0.0,0.0,0.0,0.0,1.0
1,0.00001,0.730159,0.0,0.517949,0.394737,0.017934,0.014453,1.0,0.0,0.0,0.0,1.0
2,0.00002,0.624003,0.0,0.564103,0.284211,0.017316,0.012647,1.0,0.0,0.0,0.0,0.0
3,0.00003,0.528455,1.0,0.584615,0.378947,0.018553,0.015357,0.0,0.0,0.0,0.0,1.0
4,0.00004,0.516918,0.0,0.517949,0.242105,0.015461,0.011743,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0.99994,0.653659,1.0,0.579487,0.347368,0.016698,0.013550,0.0,0.0,1.0,0.0,1.0
69996,0.99996,0.913899,0.0,0.528205,0.610526,0.017934,0.014453,0.5,0.5,0.0,0.0,1.0
69997,0.99997,0.640186,1.0,0.656410,0.500000,0.020408,0.014453,1.0,0.0,0.0,1.0,0.0
69998,0.99999,0.900736,0.0,0.553846,0.326316,0.017625,0.013550,0.0,0.5,0.0,0.0,0.0


Так лучше

In [63]:
df.describe()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,0.499729,0.671379,0.349571,0.560817,0.337925,0.017243,0.015052,0.183436,0.113229,0.088129,0.053771,0.803729
std,0.288516,0.191038,0.476838,0.042103,0.075767,0.009525,0.017026,0.340125,0.286135,0.283484,0.225568,0.397179
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.250070,0.531630,0.000000,0.533333,0.289474,0.016698,0.013550,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.500020,0.689508,0.000000,0.564103,0.326316,0.016698,0.013550,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.748900,0.815254,1.000000,0.589744,0.378947,0.017934,0.014453,0.500000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Распределение близкое к равномерному. Это значит, что мы можем смело применять модели без необходимости доработки.

In [64]:
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((49000, 12), (21000, 12), (49000,), (21000,))

In [73]:
%%time
rfc = RandomForestClassifier(
    n_estimators=500,
    criterion='gini',
    max_depth=5
)
gbc = GradientBoostingClassifier(
    n_estimators=500,
    max_depth=5
)
lrc = LogisticRegression(
    max_iter=500
)

model_list = [lrc, rfc, gbc]

results = pd.DataFrame(
    {'model':[],
     'precision':[],
     'recall':[],
     'roc_auc':[],
     'f_score': [],
     'accuracy': []})

for model in model_list:
    cv_data = cross_validate(
        model,
        df,
        target,
        scoring=('precision', 'recall', 'f1', 'roc_auc', 'accuracy'),
        cv=3,
        n_jobs=-1
    )
    results = results.append({'model':f'{model.__class__.__name__}',
                        'precision':cv_data['test_recall'].mean(),
                        'recall':cv_data['test_precision'].mean(),
                        'roc_auc':cv_data['test_roc_auc'].mean(),
                        'f_score': cv_data['test_f1'].mean(),
                        'accuracy': cv_data['test_accuracy'].mean()},
                        ignore_index=True)
results

Wall time: 40.3 s


,model,precision,recall,roc_auc,f_score,accuracy
0,LogisticRegression,0.618057,0.655820,0.703162,0.636264,0.647014
1,RandomForestClassifier,0.497875,0.758073,0.793733,0.492418,0.648859
2,GradientBoostingClassifier,0.502416,0.541423,0.658508,0.409852,0.535929


По результатам кросс-валидации можно сказать, что явным лидером при выбранных гиперпараметрах стал СЛУЧАЙНЫЙ ЛЕС.<br>
Исходя из датасета, мы скорее всего определяем есть ли у человека проблемы с сердцем.<br> * Нам очень важно растить метрику RECALL, так как лучше здорового определить к врачу, чем пропустить больного и он умрет.<br> * Метрика PRECISION нас мало интересует, так как показывает сколько больных мы идентифицировали верно. То есть шанс пропуска больного не минимизмруется.<br> * Метрика ROC_AUC показывает насколько верно модель классифицирует наблюдения. Нас вполне устривает показатель 0,79, но хотелось бы 0,99)<br> * Метрика F1 в нашем случае общая, то есть с коэффициентом лямбда=0. Если мы сместим ее значение в сторону ПОЛНОТЫ, то она сдвинется в нужную нам сторону.<br> * Метрика ACCURACY говорит, что мы верно классифицировали 65% наблюдений. Ее желательно нарастить хотя бы до 80-85% сохранив или увеличив ПОЛНОТУ<br><br>Для данной модели нужно подобрать гиперпараметры, при которых будет максимизирована метрика RECALL и ACCURACY. Для этого обычно используется grid_search.